In [1]:
import numpy as np
from nltk.stem import PorterStemmer
import re
import os

In [2]:
def preprocessing(document):
    tokens = document.split() 
    lowercase_tokens = [token.lower() for token in tokens] 

    stopwords = set()
    with open('./stopwords.txt', 'r', encoding='utf-8') as stopword_file:
        stopwords = set(stopword_file.read().splitlines())
    filtered_tokens = [token for token in lowercase_tokens if token not in stopwords]

    filtered_tokens_without_endings = [re.sub(r'[,.!?"@()%`\':;{}$&*-]+', '', token) for token in filtered_tokens]
    filtered_tokens_without_endings = [token for token in filtered_tokens_without_endings if token != '']

    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens_without_endings]
    stemmed_tokens = [token for token in stemmed_tokens if token not in stopwords]

    tokens = [token for token in stemmed_tokens if not token.isdigit() and len(token) > 1]
    
    return tokens

In [3]:
def tokenize(documents):
    dictionary_tokens = dict()
    for document in documents:
        stemmed_tokens = preprocessing(document)

        stemmed_tokens = list(set(stemmed_tokens))

        # count df
        for word in stemmed_tokens:
            if word in dictionary_tokens:
                dictionary_tokens[word] += 1
            else:
                dictionary_tokens[word] = 1
    
    sorted_dictionary = {k: v for k, v in sorted(dictionary_tokens.items())}
    
    return sorted_dictionary

In [4]:
def calculate_tf_idf(document, dictionary, i):
    tokens = preprocessing(document)
    tf = dict()
    for token in tokens:
        if token in tf:
            tf[token] += 1
        else:
            tf[token] = 1
    
    # calculate tf-idf
    tf_idf = dict()
    for token in tf:
        tf_idf[token] = tf[token] * np.log10(1095/dictionary[token])

    # calculate unit vector
    tf_idf_length = np.linalg.norm(list(tf_idf.values()))
    tf_idf_unit_vector = {token: tf_idf_value / tf_idf_length for token, tf_idf_value in tf_idf.items()}

    return tf_idf_unit_vector

In [5]:
def consine_similarity(d1, d2, tf_idf_matrix):
    unit_vector_d1 = tf_idf_matrix[d1]
    unit_vector_d2 = tf_idf_matrix[d2]

    dot_product = sum(unit_vector_d1[token] * unit_vector_d2[token] for token in unit_vector_d1 if token in unit_vector_d2)
    
    norm_x = np.linalg.norm(list(unit_vector_d1.values()))
    norm_y = np.linalg.norm(list(unit_vector_d2.values()))
    
    similarity = dot_product / (norm_x * norm_y)

    return similarity

In [6]:
class MaxHeap:
    def __init__(self):
        self.heap = []

    def insert(self, item):
        self.heap.append(item)
        self._heapify_up(len(self.heap) - 1)

    def delete_max(self):
        if len(self.heap) == 0:
            return None
        max_item = self.heap[0]
        self.heap[0] = self.heap[-1]
        self.heap.pop()
        self._heapify_down(0)
        return max_item

    def _heapify_up(self, index):
        while index > 0:
            parent_index = (index - 1) // 2
            if self.heap[index][0] > self.heap[parent_index][0]:
                self.heap[index], self.heap[parent_index] = self.heap[parent_index], self.heap[index]
                index = parent_index
            else:
                break

    def _heapify_down(self, index):
        child_index = 2 * index + 1
        while child_index < len(self.heap):
            max_child_index = child_index
            if child_index + 1 < len(self.heap) and self.heap[child_index + 1][0] > self.heap[child_index][0]:
                max_child_index = child_index + 1

            if self.heap[index][0] < self.heap[max_child_index][0]:
                self.heap[index], self.heap[max_child_index] = self.heap[max_child_index], self.heap[index]
                index = max_child_index
                child_index = 2 * index + 1
            else:
                break


In [7]:
def hac_single_link(C, K, document_number):
    heap = MaxHeap()
    for i in range(1, document_number + 1):
        for j in range(i + 1, document_number + 1):
            heap.insert((C[i][j], (i, j)))

    clusters = {i: [i] for i in range(1, document_number + 1)}
    remaining_clusters = set(clusters.keys())

    while len(remaining_clusters) > K:
        max_similarity, (i, j) = heap.delete_max()

        if i not in remaining_clusters or j not in remaining_clusters:
            continue

        clusters[i].extend(clusters[j])
        del clusters[j]
        remaining_clusters.remove(j)

        for k in remaining_clusters:
            if k != i:
                new_similarity = max(C[i][k], C[j][k])
                C[i][k] = C[k][i] = new_similarity
                heap.insert((new_similarity, (i, k)))

    return clusters


In [8]:
document_number = 1095

documents = []

# read all documents
for i in range(1, document_number + 1):
    with open(f"./data/{i}.txt", "r", encoding="utf-8") as file:
        text = file.read()
        documents.append(text)

# tokenize all documents
dictionary = tokenize(documents)

# calculate tf-idf for all documents
tf_idf_matrix = []
tf_idf_matrix.append({})
for i in range(1, 1096):
    tf_idf_matrix.append(calculate_tf_idf(documents[i - 1], dictionary, i))

C = np.zeros((document_number + 1, document_number + 1))

for i in range(1, document_number + 1):
    for j in range(1, document_number + 1):
        C[i][j] = consine_similarity(i, j, tf_idf_matrix)

K_values = [8, 13, 20]
clusters_results = {}

for K in K_values:
    clusters_results = hac_single_link(C, K, document_number)
    with open(f'./{K}.txt', 'w') as file:
        for cluster in clusters_results.values():
            for document in cluster:
                file.write(f'{document} \n')
            file.write('\n')